In [5]:
import pandas as pd
import sys
import time

# Adicionando o caminho do diretório ao sys.path
sys.path.append(r"C:\Users\Daniel\Desktop\comparativo_frete")
from Scrap_minha_casa import executa_compara_frete_minha_casa_solar
from Scrap_neosolar import executa_compara_frete_neosolar
#from Scrap_energie_total import executa_compara_frete_energie_solar


In [ ]:
# 33338 -painel 160W 
# 480 - Bomba Anauge P 100
# 13019- Bateria MOura
# 34405 - Painel Ztron 340W
# 31923  - Inversor Senoidal 1000W
product_ids_neo = ['480', '13019', '34405', '31923']
neo = executa_compara_frete_neosolar(product_ids_neo)

# 267779 -- Painel Resul 155W
# 265164 -- Bateria Moura
# 264790 -- Bomba P100
# 268645 -- Painel 340W Policristalino Half-Cell
# 269146 -- Iversor Senoidal 1000W - https://www.minhacasasolar.com.br/inversor-de-tensao-senoidal-1000w-12v-220v-hayonik-psw121-82646
product_ids_mcs = [265164, 264790, 268645, 269146]
mcs = executa_compara_frete_minha_casa_solar(product_ids_mcs)

In [7]:
def trata_custo_mcs(df):
    df['SKU'] = df['SKU'].astype(str)
    df =  df[df['Transportadora'] != 'Retirada CD - Minha Casa Solar']

    df['Custo'] = df['Custo'].str.replace(',', '')
    df['Custo'] = df['Custo'].str.replace('.', '').astype(float)
    df['Custo'] = df['Custo']/100

    return df

def trata_custo_neo(df):
    df['SKU'] = df['SKU'].astype(str)
    df =  df[df['Transportadora'] != 'Retirada CD - Minha Casa Solar']

    df['Custo'] = df['Custo'].str.replace(r'R\$', '', regex=True)  # Remove 'R$'
    df['Custo'] = df['Custo'].str.replace(r'\xa0', '', regex=True)  # Remove '\xa0'
    df['Custo'] = df['Custo'].str.replace(',', '.')  # Substitui vírgula por ponto e converte para float
    df['Custo'] = df['Custo'].str.replace('.', '')
    df['Custo'] = df['Custo'].astype(float)
    df['Custo'] = df['Custo']/100

    return df

In [ ]:
neo = trata_custo_neo(neo)
mcs = trata_custo_mcs(mcs)

In [27]:
neo.to_excel(rf"N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\Acompanhamento_Entregas\Teste Unificação\DexPara_2024\Comparativo_bi_operacoes\Comparativo_frete_neo_{time.strftime('%Y-%m-%d')}.xlsx", index=False)
mcs.to_excel(rf"N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\Acompanhamento_Entregas\Teste Unificação\DexPara_2024\Comparativo_bi_operacoes\Comparativo_frete_mcs_{time.strftime('%Y-%m-%d')}.xlsx", index=False)

In [ ]:
neo.head(5)

In [11]:
def calculamedia(tes):
  menor_prazo_menor_custo = acha_menor_custo_menor_prazo(tes)

  media = menor_prazo_menor_custo.groupby(['SKU','Empresa','UF']).agg(media_preço = ('Custo', 'mean'), media_prazo = ('Prazo', 'mean')).reset_index()

  return media

def acha_menor_custo_menor_prazo(tes):

  # Agrupamos por 'UF' e 'SKU', e para cada grupo, pegamos a linha com o menor custo
  min_custo_prazo_do_custo = tes.loc[tes.groupby(['SKU','CEP','UF'])['Custo'].idxmin()]
  
  return min_custo_prazo_do_custo


In [12]:
dim_mcs = calculamedia(mcs)
dim_neo = calculamedia(neo)

In [13]:
def classifica_produto(df):
    # Listas de SKUs para comparação
    bomba_skus = ['480', '264790']
    bateria_skus = ['13019', '265164']
    Painel_155 = ['33338', '267779']
    Painel_340 = ['34405','268645'] 
    Inversor_Senoidal_100W = ['31923','269146']

    # Função para classificar o produto com base no SKU
    def classify_product(sku):
        if sku in bomba_skus:
            return 'Bomba Solar P100'
        elif sku in bateria_skus:
            return 'Bateria Moura 220aH'
        elif sku in Painel_155:
            return 'Painel 155W'
        elif sku in Painel_340:
            return 'Painel 340W Ztroon'
        elif sku in Inversor_Senoidal_100W:
            return 'Inversor Senoidal 1000W'

    # Aplicar a função para criar a nova coluna 'Produto'
    df['Produto'] = df['SKU'].apply(classify_product)

    return df

In [14]:
dim_neo = classifica_produto(dim_neo)
dim_mcs = classifica_produto(dim_mcs)

In [15]:
def calcular_diferenca_e_porcentagem(df1, df2):
    # Passo 1: Mesclar os DataFrames com base na coluna 'UF'
    merged_df = pd.merge(df1, df2, on='UF', how='inner')

    # Passo 2: Subtrair os valores das colunas 'media_preço_x' e 'media_preço_y'
    merged_df['Dif_Preço'] = merged_df['media_preço_x'] - merged_df['media_preço_y']
    merged_df['Dif_Prazo'] = merged_df['media_prazo_x'] - merged_df['media_prazo_y']

    # Passo 3: Calcular a diferença em porcentagem e formatar as colunas
    merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2)
    merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2)
    #merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2).map("{:.2f}%".format)
    #merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2).map("{:.2f}%".format)

    return merged_df

In [ ]:
wdwdwd = pd.merge(dim_neo, dim_mcs, on=['Produto', 'UF'], how='inner')
wdwdwd.head(5)

In [17]:
def calcular_diferenca_e_porcentagem(df1, df2):
    # Passo 1: Mesclar os DataFrames com base na coluna 'UF'
    merged_df = pd.merge(df1, df2, on=['Produto', 'UF'], how='inner')

    # Passo 2: Subtrair os valores das colunas 'media_preço_x' e 'media_preço_y'
    merged_df['Dif_Preço'] = merged_df['media_preço_x'] - merged_df['media_preço_y']
    merged_df['Dif_Prazo'] = merged_df['media_prazo_x'] - merged_df['media_prazo_y']

    # Passo 3: Calcular a diferença em porcentagem e formatar as colunas
    merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2)
    merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2)
    #merged_df['%Dif_Preço'] = ((merged_df['Dif_Preço'] / merged_df['media_preço_x']) * 100).round(2).map("{:.2f}%".format)
    #merged_df['%Dif_Prazo'] = ((merged_df['Dif_Prazo'] / merged_df['media_prazo_x']) * 100).round(2).map("{:.2f}%".format)

    return merged_df

In [ ]:
regioes = pd.read_excel(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\Tabelas dimensão\D_Regiões e UF.xlsx")
regioes = regioes[['Região', 'UF']]
regioes.head(5)

In [ ]:
# chama a função para calcular a diferença e porcentagem
comparativo_semi = calcular_diferenca_e_porcentagem(dim_neo,dim_mcs)
comparativo_semi = pd.merge(comparativo_semi, regioes, on=['UF'], how='inner')
comparativo_semi.head(5)

In [ ]:
# Agrupando em função da região e produto
comparativo = comparativo_semi.groupby(['Região', 'Produto'])[['%Dif_Preço', '%Dif_Prazo']].mean().reset_index()
comparativo = comparativo.sort_values(by='Produto', ascending=True)
comparativo['%Dif_Preço'] = comparativo['%Dif_Preço'].round(2)
comparativo['%Dif_Prazo'] = comparativo['%Dif_Prazo'].round(2)
comparativo

In [ ]:
def style_and_round(df):
    # Arredondar as colunas %Dif_Preço e %Dif_Prazo para 2 casas decimais
    df['%Dif_Preço'] = df['%Dif_Preço'].round(2)
    df['%Dif_Prazo'] = df['%Dif_Prazo'].round(2)
    
    # Função para colorir as células com base nos valores
    def colorize(val):
        if val < 0:
            return 'background-color: green; color: white'
        else:
            return 'background-color: red; color: white'
    
    # Aplicar a função de formatação ao DataFrame
    return df.style.applymap(colorize, subset=['%Dif_Preço', '%Dif_Prazo'])


# Aplicar a função para arredondar e colorir
styled_comparativo = style_and_round(comparativo)

# Exibir o DataFrame estilizado
styled_comparativo

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Função para aplicar a coloração
def color_negative_red(val):
    color = 'red' if val > 0 else 'green'
    return f'color: {color}'

def color_negative_reds(df):
    # Aplica a coloração apenas nas colunas de interesse
    return df.style.applymap(color_negative_red, subset=['%Dif_Preço', '%Dif_Prazo'])

# Exibindo a tabela estilizada
styled_df = color_negative_reds(comparativo)
styled_df
